In [1]:
#|echo: false
import pandas as pd, numpy as np, matplotlib.pyplot as plt, altair as alt, pytz
from fastcore.all import *
from datetime import datetime, timedelta

## Get data

In [2]:
#|echo: false
print(f'Last execution time: {datetime.now(pytz.timezone("America/Lima")).strftime("%d/%m/%Y %T")}')

Last execution time: 20/07/2023 05:46:26


In [3]:
#|code-summary: Products type filter
explore_types = ['frutas', 'lacteos', 'verduras', 'embutidos', 'panaderia', 'desayuno', 'congelados', 'abarrotes',
                 'aves', 'carnes', 'pescados']

In [4]:
#|code-summary: Data table
path = Path('../../output')
csv_files = L(path.glob('*.csv')).filter(lambda o: os.stat(o).st_size>0)
pat_store = re.compile('(.+)\_\d+')
pat_date = re.compile('.+\_(\d+)')
df = (
    pd.concat([pd.read_csv(o).assign(store=pat_store.match(o.stem)[1], date=pat_date.match(o.stem)[1])
               for o in csv_files], ignore_index=True)
    .pipe(lambda d: d.assign(
        name=d.name.str.lower()+' ('+d.store+')',
        sku=d.id.where(d.sku.isna(), d.sku).astype(int),
        date=pd.to_datetime(d.date)
    ))
    .drop('id', axis=1)
    .loc[lambda d: d.category.str.contains('|'.join(explore_types))]
    # Filter products with recent data
    .loc[lambda d: d.name.isin(d.groupby('name').date.max().loc[ge(datetime.now()-timedelta(days=30))].index)]
    # Filter empty prices
    .loc[lambda d: d.price>0]
)
print(df.shape)
df.sample(3)

(551115, 8)


,brand,uri,name,price,category,store,date,sku
1463344,Tigo,https://www.metro.pe/yogurt-mix-tigo-vainilla-...,yogurt mix tigo vainilla con grageas de chocol...,4.20,https://www.metro.pe/lacteos/yogures/yogurt-co...,metro,2023-07-20,12840
541031,KELLOGG S,https://www.plazavea.com.pe/cereal-kelloggs-mu...,cereal kelloggs musli cosecha roja caja 300gr ...,30.19,https://www.plazavea.com.pe/desayunos,plaza_vea,2023-06-12,14901
793358,BON BEEF,https://www.plazavea.com.pe/hamburguesa-bon-be...,hamburguesa bon beef 100g bolsa 10un (plaza_vea),21.60,https://www.plazavea.com.pe/congelados,plaza_vea,2023-05-15,10010532


## Top changes (ratio)

In [5]:
top_changes = (df
 # Use last 30 days of data to compare prices
 .loc[lambda d: d.date>=(datetime.now()-timedelta(days=30))]
 .sort_values('date')
 # Get percentage change
 .assign(change=lambda d: d
     .groupby(['store','sku'], as_index=False)
     .price.transform(lambda d: (d-d.shift())/d.shift())
 )
 .groupby(['store','sku'], as_index=False)
 .agg({'price':'last', 'change':'mean', 'date':'last'})
 .rename({'price':'last_price', 'date':'last_date'}, axis=1)
 .dropna()
 .loc[lambda d: d.last_date==d.last_date.max()]
 .loc[lambda d: d.change.abs().sort_values(ascending=False).index]
)
top_changes.head(3)

,store,sku,last_price,change,last_date
8145,plaza_vea,10171150,16.99,0.409529,2023-07-20
5869,plaza_vea,12722,14.99,0.370113,2023-07-20
7470,plaza_vea,10023355,5.90,0.300752,2023-07-20


In [6]:
def plot_changes(df_changes, title):
    selection = alt.selection_point(fields=['name'], bind='legend')
    dff = df_changes.drop('change', axis=1).merge(df, on=['store','sku'])
    return (dff
     .pipe(alt.Chart)
     .mark_line(point=True)
     .encode(
         x='date',
         y='price',
         color=alt.Color('name').scale(domain=sorted(dff.name.unique().tolist())),
         tooltip=['name','price','last_price']
     )
     .add_params(selection)
     .transform_filter(selection)
     .interactive()
     .properties(width=650, title=title)
     .configure_legend(orient='top', columns=3)
    )

In [7]:
top_changes.head(10).pipe(plot_changes, 'Top changes')

alt.Chart(...)

In [8]:
(top_changes
 .sort_values('change')
 .head(10)
 .pipe(plot_changes, 'Top drops')
)

alt.Chart(...)

In [9]:
(top_changes
 .sort_values('change')
 .tail(10)
 .pipe(plot_changes, 'Top increases')
)

alt.Chart(...)

## Top changes (absolute values)

In [10]:
top_changes_abs = (df
 # Use last 30 days of data to compare prices
 .loc[lambda d: d.date>=(datetime.now()-timedelta(days=30))]
 .sort_values('date')
 # Get percentage change
 .assign(change=lambda d: d
     .groupby(['store','sku'], as_index=False)
     .price.transform(lambda d: (d-d.shift()).iloc[-1])
 )
 .groupby(['store','sku'], as_index=False)
 .agg({'price':'last', 'change':'mean', 'date':'last'})
 .rename({'price':'last_price', 'date':'last_date'}, axis=1)
 .dropna()
 .loc[lambda d: d.last_date==d.last_date.max()]
 .loc[lambda d: d.change.abs().sort_values(ascending=False).index]
)
top_changes_abs.head(3)

,store,sku,last_price,change,last_date
5656,plaza_vea,10083,49.43,-16.47,2023-07-20
5655,plaza_vea,10081,49.43,-16.47,2023-07-20
5697,plaza_vea,10294,47.18,-15.72,2023-07-20


In [11]:
top_changes_abs.head(10).pipe(plot_changes, 'Top changes')

alt.Chart(...)

In [12]:
(top_changes_abs
 .sort_values('change')
 .head(10)
 .pipe(plot_changes, 'Top drops')
)

alt.Chart(...)

In [13]:
(top_changes_abs
 .sort_values('change')
 .tail(10)
 .pipe(plot_changes, 'Top increases')
)

alt.Chart(...)

## Search specific products

In [14]:
#|echo: false
#|output: false
names = df.name[df.name.str.contains(r'(?=.*pollo)(?=.*entero).*') &
                ~df.name.str.contains(r'marinado|aderezo')].unique().tolist()
names

['pollo entero light  x kg (metro)',
 'pollo entero fresco metro x kg (metro)',
 'pollo entero\xa0artisan\xa0libre de antibióticos x kg (plaza_vea)']

In [15]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Pollo')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)

In [16]:
#|echo: false
#|output: false
names = df.name[df.name.str.contains(r'palta') &
                ~df.name.str.contains(r'shampoo|humectante|vino|salsa|acondicionador|aceite')].unique().tolist()
names

['palta madura cremosita x kg (metro)',
 'palta fuerte metro x kg (metro)',
 'palta hass x kg (metro)',
 'palta hass madura la caserita empaque 500g (plaza_vea)',
 "palta fuerte bell's madura (plaza_vea)",
 'palta fuerte (plaza_vea)',
 'palta hass madura natifrut 500g (metro)']

In [17]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Palta')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)

In [18]:
#|echo: false
#|output: false
names = df.name[df.name.str.contains(r'(?=.*aceite)(?=.*vegetal)(?=.*900).*') &
                ~df.name.str.contains(r'atun|atún|pack|filete|caballa|tacos|sardinas')].unique().tolist()
names

['aceite vegetal primor clásico 900ml (metro)',
 'aceite vegetal cocinero 900ml (metro)',
 'aceite vegetal cuisine&co botella 900 ml (metro)',
 'aceite vegetal máxima 900ml (metro)',
 'aceite vegetal deleite premium 900ml (metro)',
 'aceite vegetal primor premium 900ml (metro)',
 'aceite vegetal metro 900ml (metro)',
 'aceite vegetal nicolini 900ml (metro)',
 'aceite vegetal cocinero botella 900ml (plaza_vea)',
 'aceite vegetal primor clásico botella 900ml (plaza_vea)',
 'aceite vegetal nicolini botella 900ml (plaza_vea)',
 'aceite vegetal deleite botella 900ml (plaza_vea)',
 'aceite vegetal cil botella 900ml (plaza_vea)',
 'aceite vegetal del cielo botella 900ml (plaza_vea)',
 'aceite vegetal primor premium botella 900ml (plaza_vea)',
 "aceite vegetal bell's botella 900ml (plaza_vea)"]

In [19]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Aceite')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)

In [20]:
#|echo: false
#|output: false
names = df.name[
    df.name.str.contains(r'(?=.*harina)(?=.*1kg).*')
#     & ~df.name.str.contains(r'atun|atún|pack|filete|caballa|tacos|sardinas')
].unique().tolist()
names

['harina preparada favorita 1kg (metro)',
 'harina preparada molitalia 1kg (metro)',
 'harina preparada blanca flor 1kg (metro)',
 'harina sin preparar favorita 1kg (metro)',
 'harina sin preparar blanca flor 1kg (metro)',
 'harina sin preparar molitalia 1kg (metro)',
 'harina sin preparar nicolini 1kg (metro)',
 'harina de maíz amarillo p.a.n. precocida bolsa 1kg (plaza_vea)',
 'harina molitalia preparada bolsa 1kg (plaza_vea)',
 'harina sin preparar favorita paquete 1kg (plaza_vea)',
 'harina preparada grano de oro paquete 1kg (plaza_vea)',
 'harina de trigo nicolini sin preparar bolsa 1kg (plaza_vea)',
 'harina sin preparar blanca flor bolsa 1kg (plaza_vea)',
 "harina bell's de trigo preparada bolsa 1kg (plaza_vea)",
 'harina de trigo nicolini preparada bolsa 1kg (plaza_vea)',
 'harina selecta molitalia bolsa 1kg (plaza_vea)',
 'harina la casa marimel integral bolsa 1kg (plaza_vea)',
 'harina de trigo sin preparar molitalia sin preparar bolsa 1kg (plaza_vea)',
 'harina de trigo prep

In [21]:
(df
 .loc[df.name.isin(names)]
 .pipe(alt.Chart)
 .mark_line(point=True)
 .encode(x='date', y='price', color='name', tooltip=['name','price'])
 .properties(width=650, title='Aceite')
 .interactive()
 .configure_legend(orient='top', columns=3)
)

alt.Chart(...)